# Estimate size of migration

The object of this migration is to migrate MS Access databases to comma separated values. In the past, conversions of table data from MS Access databases have been done; database metadata have never been converted. These last two statements are both presumptions: the conversions were done by hand and provenance has not been recorded.

In this notebook we connect to the sql database and select candidates for migration.

- According to the sql database there are 4786 *.mdb files and 136 *.accdb files.
- That makes a total of 4922 Access db's. 
- Only 7 of them have a proper media type (application/x-msaccess) in the database.
- The 4922 Access db's take a total size of 39 Gb.
- The 4922 Access db's are distributed over 2316 datasets.
- Most datasets have 1 database, there is one dataset with 1037 databases.

### Create a symlink to a spacy workspace

In [1]:
!df -h /Volumes/office-space

Filesystem                                Size   Used  Avail Capacity     iused       ifree %iused  Mounted on
//henkb@do03.ia.knaw.local/office-space   48Ti   35Ti   13Ti    73% 37405043326 14089671168   73%   /Volumes/office-space


In [2]:
%%bash
dir=explore
if [ ! -e $dir ]; then
    ln -s /Volumes/office-space/TA/2018/migration/migr201801/explore $dir
    echo created symlink to $dir
fi

### Set up a connection

In [3]:
import getpass
secret = getpass.getpass()

In [4]:
import psycopg2
import json

with open('../../../src/config.json') as f:
    conf = json.load(f)
    
conn_str = "host={} port={} dbname={} user={} password={}".format(conf['host'], conf['port'], conf['database'], conf['user'], secret)
conn = psycopg2.connect(conn_str)

In [5]:
import pandas as pd
pd.options.display.max_colwidth = 0

sql = """
SELECT version();
"""
pd.read_sql(sql, con=conn)

,version
0,"PostgreSQL 8.4.20 on x86_64-redhat-linux-gnu, compiled by GCC gcc (GCC) 4.4.7 20120313 (Red Hat 4.4.7-18), 64-bit"


### How many

In [6]:
pd.reset_option('max_colwidth')

sql = """
SELECT count(*) from easy_files
WHERE filename like '%.mdb'
"""
pd.read_sql(sql, con=conn)

,count
0,4786


In [7]:
sql = """
SELECT count(*) from easy_files
WHERE filename like '%.accdb'
"""
pd.read_sql(sql, con=conn)

,count
0,136


In [8]:
sql = """
select count(*), mimetype from easy_files
where filename like '%.accdb' or filename like '%.mdb'
group by mimetype;
"""
pd.read_sql(sql, con=conn)

,count,mimetype
0,4915,application/octet-stream
1,7,application/x-msaccess


In [9]:
(4786 + 136) == 4922 & 4922 == (4915 + 7)

True

According to the database there are **4786** `*.mdb` files and **136** `*.accdb` files. Makes a total of **4922** files. Only **7** of them have a proper media type (`application/x-msaccess`) in the database.

### Select candidates for migration

In [10]:
sql = """
select * from easy_files
where filename like '%.mdb' or filename like '%.accdb';
"""
dfm = pd.read_sql(sql, con=conn)
dfm.set_index('pid')
dfm.to_csv("explore/all_msdb.csv", index=False)
dfm.shape

(4922, 11)

In [11]:
dfm['size'].sum()/2**30

39.31891160644591

In [12]:
sd = dfm.groupby(['dataset_sid']).size()
sd.describe()

count    2316.000000
mean        2.125216
std        25.717738
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max      1037.000000
dtype: float64

The **4922** databases take a total size of **39 Gb**. The **4922** databases are distributed over **2316** datasets. Most datasets have **1** database, there is one dataset with **1037** databases.

### Which ones have already been converted?

The amount of datasets that contain Access databases **and** `csv` files gives a clue to how many have been converted. (*But not realy!*) The only thing we can derive savely is the number of datasets that **not** have been converted (because they have no `csv` files). It appeared that at one time there were **160** datasets with a database and no `csv` files.

The next query with a subselect takes 6 hours. So we will not repeat it again..

In [13]:
sql = """
select * from easy_files
where dataset_sid in 
    (select dataset_sid from easy_files
    where filename like '%.mdb' or filename like '%.accdb')
    and filename like '%.csv';
"""
# dfc = pd.read_sql(sql, con=conn)
# dfc.to_csv("explore/db_csv.csv")

In [14]:
conn.close()